<a href="https://colab.research.google.com/github/Coolshegs10/Speech-to-Text-App/blob/main/Copy_of_New_Speech_to_Text_Project_Main_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Install the Necessary Packages

In [ ]:
# @title
# Upload the raw audio file
# uploaded = files.upload()

In [ ]:
# @title
# Access the uploaded file
# file_name = next(iter(uploaded))

In [ ]:
# ipd.Audio(new_audio, rate=sr)

In [ ]:
# IT STARTS HERE

!pip uninstall transformers  -y
!pip install transformers


Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.9 MB/s eta 0:00:00


In [ ]:
import soundfile as sf

In [ ]:
!pip install --upgrade transformers


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
pip show transformers

Name: transformers
Version: 4.36.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
pip install transformers --upgrade-strategy eager

In [ ]:
pip uninstall transformers && pip install transformers

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.36.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? 

In [ ]:
!python3 -m pip cache purge

In [ ]:
!pip install torch_sdpa

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import torch

# Check and set the device (GPU if available, CPU otherwise)
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import soundfile as sf

# Let's import necessary audio processing libraries
import os
import torchaudio

# Let's specify the device based on availability of GPU/CPU, enabling an optimized processing based on the available hardware
processing_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Method 1: Wav2Vec2.0 Model

In [ ]:
# The major models under Wav2Vec2.0 include:
# wav2vec2_model_name = "facebook/wav2vec2-base-960h" # 360MB
# wav2vec2_model_name = "facebook/wav2vec2-large-960h-lv60-self" # pretrained 1.26GB
wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-english" # English-only, 1.26GB
# wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-arabic" # Arabic-only, 1.26GB
# wav2vec2_model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-spanish" # Spanish-only, 1.26GB


# my project focused on a Enghlish speech to text conversion, hence:
wav2vec2_processor = Wav2Vec2Processor.from_pretrained(wav2vec2_model_name)
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained(wav2vec2_model_name).to(processing_device)

In [ ]:
# Supporting Libariries for Audio files
import librosa # librosa is a Python package for music and audio processing
import librosa.display
import IPython.display as ipd  # to play the audio signal
import matplotlib.pyplot as plt #plot the audio signal in time domain

In [ ]:
import requests
import librosa
import io
import tempfile

# audio_url = "http://www.fit.vutbr.cz/~motlicek/sympatex/f2bjrop1.0.wav"
audio_url = "http://www.fit.vutbr.cz/~motlicek/sympatex/f2bjrop1.0.wav"


# Download the audio file
response = requests.get(audio_url)

# Create a temporary file and write the audio data
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    temp_file.write(response.content)
    temp_file.seek(0)

    # Load the audio file using librosa
    x, sr = librosa.load(temp_file.name)

# Play the audio using IPython display
ipd.Audio(x, rate=sr)


In [ ]:
# Let's calculate the time domain analaysis of the audio file

plt.figure(figsize=(14, 5)) # this is the variable to change the rnage on the x and y axis
plt.grid(True) # now the grid is enabled

#  Let's plot the audio signal
librosa.display.waveshow(x) #it takes one argument
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.title("Time Domain Analysis of the Audio file")


In [ ]:
# Let's determine the duration of the audio file
y=len(x) #number of sample x
print(y)
y1=sr #sampling frequency sr
print(y1)
y=len(x)
duration_of_sound=y/y1
print(duration_of_sound, "seconds")

In [ ]:
# Let's load the audio file
speech, sr = torchaudio.load(audio_url)
speech = speech.squeeze()
sr, speech.shape

In [ ]:
# Now, let's resample the audio sampling form it's initial rate to 1600
resampler = torchaudio.transforms.Resample(sr, 16000)
speech = resampler(speech)
speech.shape

In [ ]:
# Now, let's tokenize the wav file
input_values = wav2vec2_processor(speech, return_tensors="pt", sampling_rate=16000)["input_values"].to(processing_device)
input_values.shape

In [ ]:
# Next, let's perform inference
logits = wav2vec2_model(input_values)["logits"]
logits.shape

In [ ]:
# Let's use argmax to find the predicted IDs
predicted_ids = torch.argmax(logits, dim=-1)
predicted_ids.shape

In [ ]:
# Now, let's decode the IDs to text
wav2vec2_transcription = wav2vec2_processor.decode(predicted_ids[0])
wav2vec2_transcription.lower()

# Method 2: Whisper Models

In [ ]:
!pip install --upgrade whisper

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [ ]:
# Since this is a English based STT project, we chose the English-only medium model
whisper_model_name = "openai/whisper-medium.en" # Engllish-only, ~3.06 GB

whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name).to(processing_device)

In [ ]:
# Use response.content directly without temporary file
x, sr = librosa.load(io.BytesIO(response.content), sr=sr)
input_features = whisper_processor(x, sampling_rate=16000, return_tensors="pt").input_features.to(processing_device)


In [ ]:
forced_decoder_ids = whisper_processor.get_decoder_prompt_ids(language= "english", task="transcribe")

In [ ]:
forced_decoder_ids

In [ ]:
input_features.shape

In [ ]:
predicted_ids = whisper_model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
predicted_ids.shape

In [ ]:
whisper_transcription1 = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)
whisper_transcription1

In [ ]:
whisper_transcription_2 = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=False)
whisper_transcription_2

# Part 2: System Performance Calculation  

In [ ]:
# To calculate the wer, we need to use a ground truth that will serve as the original transcript

# Here's the trancription to be used as the ground transcription
transcript = "Wanted, Chief Justice of the Massachusetts Supreme Court. In April, the SJC's current leader, Edward Hennessey, reaches the mandatory retirement age of 70, and the successor is"


In [ ]:
# Now let's preprocess our trancription to get it ready for the calculation

import re

def process_text(transcript):
  # Let's convert it to lowercase first
  transcript = transcript.lower()

  # Let's reove special characters and punctuations
  transcript = re.sub(r'[^a-zA-Z0-9\s]','', transcript)

  # let's tokenize the transcription by splitting the texts into words
  tokens = transcript.split()

  return tokens

# Let's check if it was preprocessed successfully
preprocessed_transcript = process_text(transcript)

# print the preprocessed text
print(preprocessed_transcript)

In [ ]:
type(preprocessed_transcript)

In [ ]:
print(transcript)

In [ ]:
print(preprocessed_transcript)

In [ ]:
pip install dill

In [ ]:
type(wav2vec2_transcription)

In [ ]:
type(whisper_transcription_2)

In [ ]:
from transformers import WhisperTokenizer

In [ ]:
pip install --upgrade evaluate jiwer

In [ ]:
from evaluate import load

In [ ]:
print(wav2vec2_transcription)

In [ ]:
print(whisper_transcription1)

In [ ]:
ref_words = transcript
hyp_words = wav2vec2_transcription


In [ ]:
type(whisper_transcription1)

In [ ]:
wer_metric = load("wer")

wer = wer_metric.compute(references=[ref_words], predictions=[hyp_words])

print(wer)

In [ ]:
# We need to convert the whisper_transaction to str from a list so we can perform the wer
whisper_string = ''.join(whisper_transcription1)
print(whisper_string)


In [ ]:
type(whisper_string)

In [ ]:
wer_metric = load("wer")

wer = wer_metric.compute(references=[ref_words], predictions=[whisper_string])

print(wer)

In [ ]:
!pip install pydub

# Section 3: Raw record

In [ ]:
from pydub import AudioSegment

In [ ]:
from google.colab import files

In [ ]:
from google.colab import files
from pydub import AudioSegment
from io import BytesIO

In [ ]:
# Upload the raw audio file
uploaded = files.upload()

In [ ]:

# Access the uploaded file
file_name = next(iter(uploaded))


In [ ]:

# Convert the uploaded audio file to AudioSegment
audio_data = BytesIO(uploaded[file_name])
audio_record = AudioSegment.from_file(audio_data, format="aac")


In [ ]:

# Export the audio to WAV format
new_audio = 'new_audio.wav'
audio_record.export(new_audio, format="wav")

print(f"Conversion completed. WAV file saved as {new_audio}")


In [ ]:
ipd.Audio(new_audio, rate=sr)

In [ ]:
audio_data = open('new_audio.wav', 'rb').read()

In [ ]:

# Let's determine the duration of the audio file
y=len(audio_data) #number of sample x
print(y)
y1=sr #sampling frequency sr
print(y1)
y=len(audio_data)
duration_of_sound=y/y1
print(duration_of_sound, "seconds")

In [ ]:

# Step 1: Load the audio file
import torchaudio
from torchaudio.transforms import Resample

# Load the audio file
audio_path = 'new_audio.wav'
speech, sr = torchaudio.load(audio_path)
speech = speech.squeeze()



In [ ]:
# Step 2: Resample the audio to 16,000 Hz
resampler = Resample(sr, 16000)
speech_resampled = resampler(speech)

In [ ]:
# Step 3: Prepare the audio for transcription with Whisper model
# Assuming 'whisper_processor' and 'processing_device' are initialized

# Tokenize the resampled audio
input_values = whisper_processor(speech_resampled, sampling_rate=16000, return_tensors="pt").input_features.to(processing_device)



In [ ]:
# Step 4: Perform inference (transcription) with the Whisper model
# Assuming 'whisper_model' is initialized
logits = whisper_model(input_values)["logits"]
# Process logits further if necessary to get the transcription result


In [ ]:
# import speech_recognition as sr

# recognizer = sr.Recognizer()

''' recording the sound '''

# with sr.AudioFile("./sample_audio/speech.wav") as source:
    #recorded_audio = recognizer.listen(source)
    #print("Done recording")

''' Recorgnizing the Audio '''
#try:
    #print("Recognizing the text")
    #text = recognizer.recognize_google(
            #recorded_audio,
            #language="en-US"
        #)
    #print("Decoded Text : {}".format(text))

#except Exception as ex:
    #print(ex)